In [33]:
import requests
import json
import pandas as pd
import os

def generate_query(first, skip):
  query = """
    query {
      ranking(first: %s, skip: %s) {
        items {
          id
          name
          twitter
          proposalsCount
          votesCount
          followersCount
          website
        }
      }
    }
  """ % (first, skip)
  return query


out_dir = 'cached_snapshot_data'

def get_all_deployments():
    URL = "https://hub.snapshot.org/graphql"


    has_next_page = True
    skip = 0
    while has_next_page:
        # if already cached, skip
        if os.path.exists(f'{out_dir}/snapshot_organizations_{skip}.json'):
            skip += 20
            # print('skipping', skip)
            continue
        query = generate_query(20, skip)
        print('.', skip)
        r = requests.post(URL, json={'query': query})
        data = json.loads(r.text)

        print(data)
        
        organizations = data['data']['ranking']['items']

        if not organizations:
            print('No organizations found')
            if data.get('errors'):
                print(data['errors'])
                if 'The `skip` argument must not be greater than 15000' in data['errors'][0]['message']:
                    break

        # save to out_dir
        with open(f'{out_dir}/snapshot_organizations_{skip}.json', 'w') as f:
            json.dump(organizations, f)
        
        if len(organizations) < 20:
            has_next_page = False
        skip += 20


df = get_all_deployments()

. 15020
{'errors': [{'message': 'The `skip` argument must not be greater than 15000', 'locations': [{'line': 3, 'column': 7}], 'path': ['ranking']}], 'data': {'ranking': None}}


TypeError: 'NoneType' object is not subscriptable

In [34]:
# load all cached data into a df
files = os.listdir(out_dir)
files = [f for f in files if 'snapshot_organizations' in f]
dfs = []
for f in files:
    with open(f'{out_dir}/{f}', 'r') as f:
        data = json.load(f)
    dfs.append(pd.DataFrame(data))
# reset index
df = pd.concat(dfs)
df

,id,name,twitter,proposalsCount,votesCount,followersCount,website
0,odidao.eth,ODI DAO,None,6,1,6,None
1,olodao.eth,YING TG,None,1,6,6,https://www.partybid.app/buy/0xB74BE402fEfE6bb...
2,onearth.eth,OnEarth DAO,kevinonearth999,3,6,4,https://imagineonearth.xpandrewards.xyz/
3,opyn11.eth,opyn11,None,1,5,7,None
4,partyplanningdao.eth,Party Planning DAO,None,1,1,11,None
...,...,...,...,...,...,...,...
15,moonwind.eth,Crypto Coven,None,0,0,0,None
16,mooorse.eth,mooorse,None,1,0,1,None
17,mopn.eth,MOPN,mopnxyz,0,0,1,https://mopn.xyz/
18,moreandmore.eth,moreDao,None,0,0,1,None


In [35]:
# how many have less than 10 proposals?
(df['proposalsCount'] < 10).value_counts()
# preview of events to come...

True     12975
False     2042
Name: proposalsCount, dtype: int64

In [36]:
# combine twitter and website into an 'additional' column, ignore NaNs
df['additional'] = df[['twitter', 'website']].apply(lambda x: ', '.join(x.dropna()), axis=1)
# drop twitter and website columns
df.drop(['twitter', 'website'], axis=1, inplace=True)
df

,id,name,proposalsCount,votesCount,followersCount,additional
0,odidao.eth,ODI DAO,6,1,6,
1,olodao.eth,YING TG,1,6,6,https://www.partybid.app/buy/0xB74BE402fEfE6bb...
2,onearth.eth,OnEarth DAO,3,6,4,"kevinonearth999, https://imagineonearth.xpandr..."
3,opyn11.eth,opyn11,1,5,7,
4,partyplanningdao.eth,Party Planning DAO,1,1,11,
...,...,...,...,...,...,...
15,moonwind.eth,Crypto Coven,0,0,0,
16,mooorse.eth,mooorse,1,0,1,
17,mopn.eth,MOPN,0,0,1,"mopnxyz, https://mopn.xyz/"
18,moreandmore.eth,moreDao,0,0,1,


In [37]:
# add platform = 'snapshot'
df['platform'] = 'snapshot'
# rename id to platform_id
df.rename(columns={'id': 'platform_id'}, inplace=True)
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform
0,odidao.eth,ODI DAO,6,1,6,,snapshot
1,olodao.eth,YING TG,1,6,6,https://www.partybid.app/buy/0xB74BE402fEfE6bb...,snapshot
2,onearth.eth,OnEarth DAO,3,6,4,"kevinonearth999, https://imagineonearth.xpandr...",snapshot
3,opyn11.eth,opyn11,1,5,7,,snapshot
4,partyplanningdao.eth,Party Planning DAO,1,1,11,,snapshot
...,...,...,...,...,...,...,...
15,moonwind.eth,Crypto Coven,0,0,0,,snapshot
16,mooorse.eth,mooorse,1,0,1,,snapshot
17,mopn.eth,MOPN,0,0,1,"mopnxyz, https://mopn.xyz/",snapshot
18,moreandmore.eth,moreDao,0,0,1,,snapshot


In [38]:
# add website column https://snapshot.org/#/ + platform_id
df['website'] = 'https://snapshot.org/#/' + df['platform_id']
df

,platform_id,name,proposalsCount,votesCount,followersCount,additional,platform,website
0,odidao.eth,ODI DAO,6,1,6,,snapshot,https://snapshot.org/#/odidao.eth
1,olodao.eth,YING TG,1,6,6,https://www.partybid.app/buy/0xB74BE402fEfE6bb...,snapshot,https://snapshot.org/#/olodao.eth
2,onearth.eth,OnEarth DAO,3,6,4,"kevinonearth999, https://imagineonearth.xpandr...",snapshot,https://snapshot.org/#/onearth.eth
3,opyn11.eth,opyn11,1,5,7,,snapshot,https://snapshot.org/#/opyn11.eth
4,partyplanningdao.eth,Party Planning DAO,1,1,11,,snapshot,https://snapshot.org/#/partyplanningdao.eth
...,...,...,...,...,...,...,...,...
15,moonwind.eth,Crypto Coven,0,0,0,,snapshot,https://snapshot.org/#/moonwind.eth
16,mooorse.eth,mooorse,1,0,1,,snapshot,https://snapshot.org/#/mooorse.eth
17,mopn.eth,MOPN,0,0,1,"mopnxyz, https://mopn.xyz/",snapshot,https://snapshot.org/#/mopn.eth
18,moreandmore.eth,moreDao,0,0,1,,snapshot,https://snapshot.org/#/moreandmore.eth


In [39]:
# reorder columns
df = df[['platform', 'platform_id', 'name', 'additional', 'website']]
df

,platform,platform_id,name,additional,website
0,snapshot,odidao.eth,ODI DAO,,https://snapshot.org/#/odidao.eth
1,snapshot,olodao.eth,YING TG,https://www.partybid.app/buy/0xB74BE402fEfE6bb...,https://snapshot.org/#/olodao.eth
2,snapshot,onearth.eth,OnEarth DAO,"kevinonearth999, https://imagineonearth.xpandr...",https://snapshot.org/#/onearth.eth
3,snapshot,opyn11.eth,opyn11,,https://snapshot.org/#/opyn11.eth
4,snapshot,partyplanningdao.eth,Party Planning DAO,,https://snapshot.org/#/partyplanningdao.eth
...,...,...,...,...,...
15,snapshot,moonwind.eth,Crypto Coven,,https://snapshot.org/#/moonwind.eth
16,snapshot,mooorse.eth,mooorse,,https://snapshot.org/#/mooorse.eth
17,snapshot,mopn.eth,MOPN,"mopnxyz, https://mopn.xyz/",https://snapshot.org/#/mopn.eth
18,snapshot,moreandmore.eth,moreDao,,https://snapshot.org/#/moreandmore.eth


In [40]:
# save to csv
df.to_csv('snapshot_deployments.csv', index=False)